In [1]:
from src.midi.midiToNotes import MidiProcessor
from src.piano.piano import Piano
from src.hand.hand import Hand
from src.utils import generate_finger_distribution
from src.hand.finger import Finger
from src.recorder.recorder import Recorder
from src.recorder.recorderPool import RecorderPool

midi_name = "God knows"
track_numbers = [1]
channel_number = -1
hand_range = 12
FPS = 60
# 是否要提高八度
higher_octave = False

# 初始各种文件路径
track_text = str(track_numbers[0]) if len(track_numbers) == 1 else "_".join([str(track_number) for track_number in track_numbers])
hand_recorder_file_path = f"output/hand_recorders/{midi_name}_{track_text}.hand"

# 读取midi文件，获得音高和时间轴
midi_processor = MidiProcessor(midi_name, track_numbers, channel_number=channel_number,FPS=FPS)
midi_info = midi_processor.export_midi_info()
print(midi_info)
notes_maps = midi_processor.generate_notes_map_and_messages(higher_octave)

# 初始化钢琴
piano = Piano()

# 生成手掌上手指坐标分布列表
finger_number = 5
finger_distribution = generate_finger_distribution(5)
finger_range = 12 / (max(finger_distribution) - min(finger_distribution))

# 初始化左右手,左手放在C3，右手放在C5
left_hand = Hand([Finger(0, piano.note_to_key(48)), Finger(1, piano.note_to_key(50)), Finger(2, piano.note_to_key(52)), Finger(3, piano.note_to_key(53)), Finger(4, piano.note_to_key(55))], piano, True)
right_hand = Hand([Finger(5, piano.note_to_key(72), False), Finger(6, piano.note_to_key(74), False), Finger(
    7, piano.note_to_key(76), False), Finger(8, piano.note_to_key(77), False), Finger(9, piano.note_to_key(79), False)], piano, False)

init_real_tick = 0.0
init_real_ticks = []

# 初始化recorder
recorder = Recorder(piano, [left_hand], [right_hand],
                    0, init_real_tick, init_real_ticks)

# 初始化recorderPool
pool_size = 100
recorder_pool = RecorderPool([recorder],pool_size,0)

Track 0: 
Track 1: 
Track 1, Channel 0: Acoustic Grand Piano
Track 2: 

一共有1个轨道
pitch_wheel_map 保存到了 asset/temp/pitch_wheel_map.json
messages 保存到了 asset/temp/messages.json
全曲的速度变化是:
在0轨，tick为0时，速度变为750002
在0轨，tick为480时，速度变为681816
在0轨，tick为640时，速度变为705881
在0轨，tick为1440时，速度变为750002
在0轨，tick为1920时，速度变为800000
在0轨，tick为2080时，速度变为857139
在0轨，tick为2640时，速度变为2222222
在0轨，tick为2880时，速度变为800000
在0轨，tick为4320时，速度变为857139
在0轨，tick为4640时，速度变为1090909
在0轨，tick为4800时，速度变为1200000
在0轨，tick为4960时，速度变为1200000
在0轨，tick为5760时，速度变为800000
在0轨，tick为7680时，速度变为666667
在0轨，tick为7920时，速度变为750002
在0轨，tick为8160时，速度变为857139
在0轨，tick为8400时，速度变为1333332
在0轨，tick为8640时，速度变为800000
在0轨，tick为9120时，速度变为923079
在0轨，tick为9600时，速度变为1090909
在0轨，tick为9840时，速度变为1578948
在0轨，tick为10080时，速度变为1333332
在0轨，tick为11520时，速度变为382165
在0轨，tick为13440时，速度变为372670
在0轨，tick为24960时，速度变为359281
在0轨，tick为29760时，速度变为375000
在0轨，tick为36480时，速度变为379746
在0轨，tick为46800时，速度变为372670
在0轨，tick为57600时，速度变为377357
在0轨，tick为59520时，速度变为372670
在0轨，tick为67200时，速度变为379746

In [2]:
from tqdm import tqdm

# 使用notes_map，开始迭代更新recorder，并且放入recorder_pool中
for notes_map in tqdm(notes_maps, desc="生成指法中……", unit="step"):
    recorder_pool.update_recorder_pool(
        notes_map, hand_range, finger_range, finger_distribution)

recorder_pool.export_pool_info(hand_recorder_file_path)

生成指法中……: 100%|██████████| 1284/1284 [01:13<00:00, 17.51step/s]

最优记录的熵值为：20036.75999999992
已保存至output/hand_recorders/God knows_1.hand


In [3]:
from src.animation.animator import Animator

avatar_name = 'Kinich'
avatar_info_path = f'asset/avatars/{avatar_name}.avatar'

animator = Animator(hand_recorder_file_path, avatar_info_path,piano)
animator.generate_animation_info()

已加载output/hand_recorders/God knows_1.hand 和 asset/avatars/Kinich.avatar
动画数据已保存到output/animation_recorders/God knows_1_Kinich.animation
